### Imports

In [1]:
# Back-end libs
import sqlite3
import pandas as pd
import numpy
from pathlib import Path

# Fornt-end libs
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_table

# Back-end Functions

In [2]:
def rename(dataframe):
#     col=['Date','Ref','Ab_change','Rel_change','Close','Vol','Open','High','Low','Agreement','For_sell','For_buy','For_net']
    
#     col=['Date','Ref','Ab_change','Rel_change','Close','Vol','Open','High','Low','Agreement','For_sell','For_buy']
    dataframe.columns = dataframe.iloc[0,:]
    dataframe.drop(index=0,inplace=True)
#     dataframe.drop(columns=['STT'],inplace=True)
    dataframe.reset_index(inplace=True,drop=True)
    
#     dataframe.columns = col

In [3]:
def all_files():
    return [path for path in list(Path(r'./All_prices').glob('**/*')) if '.xlsx' in str(path)]

In [4]:
def thousand_handle(ar):
    if type(ar) == str:
        if '%' in ar:
            ar = ar.replace('%','')
            ar= ar.replace(',','')
            ar = ar.replace('.','')
            ar = int(ar)/10000
        else:
            ar= ar.replace(',','')
#             ar = ar.replace('.','')
            ar = float(ar)
    return ar

In [5]:
def all_handle(dataframe):
    rename(dataframe)
    dataframe.drop(columns=[0,'STT'],inplace=True)
    dataframe.dropna(how='all',axis=0,inplace=True)
#     dataframe['Ngày'] = pd.to_datetime(dataframe['Ngày']) 
#     for col in dataframe.columns:
#         dataframe[col] = dataframe[col].map(thousand_handle)
    return dataframe

In [6]:
def sql_conn(ind):
    conn =  sqlite3.connect('./Data/%s.db'%ind)
    return conn

In [7]:
def to_sql(dataframe,stk,conn,if_existed):
    dataframe.to_sql('%s'%stk,con=conn,if_exists = if_existed)

In [8]:
def get_stk(stk):
    return pd.read_sql('select * from %s' %(stock),con=conn)
# It's better if we could add cols argurment for choosing  columns.
#     if not cols:
#         return pd.read_sql('select * from %s' %(stock),con=conn)
#     else:
#         cols.append(stk)
#         return pd.read_sql('select %s from %s' %(cols),con=conn)

In [15]:
def get_stk_name():
    f= open("D:/Simulation/All_prices/sorted_all_prices.txt","r")
    String_stk= f.read()
    stk_lis = String_stk.split(",")
    f.close()
    stk_dict = [{'value':stk_lis[i],'label':stk_lis[i]} for i in range(len(stk_lis))]
    return stk_dict

# Font-end Functions

### Plan: Crawler --> Local data --> SQLite (empty query input) --> Dash+Ngrok

#### 1/Dash + Ngrok

# - ISSUES:
## + There are no connection between back-end and front-end. Firstly, we have to read input from id='stock-id', then put them into get stk function to get data table. Then draw data table on page.
## + We could add function to draw chart from selected table.
## + The next step is to build back-end function to calculate stocks' weight and PnL. Then combine them wit price, volume data to get PnL chart of portfolio.

In [26]:
# All stock name:
opt_lis = get_stk_name()

# Dash App Innitialize
app = dash.Dash(__name__,
                external_stylesheets=[dbc.themes.CYBORG],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, innitial-scale=1.0'
                           }]
               )

# Dash App Layout
app.layout = dbc.Container([
    dbc.Row( # Page Header
        dbc.Col(
            html.H2(
                'DATA TRANFERS',
                className= 'h1'
            ),
            width= 12,
        )
    ),
    dbc.Row( # Query input
        [
            dbc.Col(
                dcc.Dropdown( # Choose which stock to tranfer to SQLite
                    id= 'stock_id',
                    options=opt_lis,
                    multi=True,
                    placeholder='Choose Stocks!'
                ), 
                width = 3
            ),
            
            dbc.Col(
                dbc.Input(
                    id= 'query',
                    debounce= True,
                    placeholder='Input Queries!'
                ),
                width = 9
            )
        ]
    )
],fluid=True)


# ----- ISSUES HERE

@app.callback(
    Output(component_id='stock_id', component_property='value'),
    Input(component_id='query', component_property='value')
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)

In [27]:
if __name__ == '__main__':
    app.run_server(port=3001, debug= True, use_reloader=False)

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
